In [1]:
import pdfplumber
import pandas as pd

# Path to the PDF file
pdf_path = 'pdfs/Bedford.pdf'

# Open the PDF file
with pdfplumber.open(pdf_path) as pdf:
    # Initialize an empty list to store the extracted data
    data = []
    # Iterate through each page
    for page_num, page in enumerate(pdf.pages):
        print(f"Page {page_num + 1}")
        # Extract the text with bounding boxes
        for element in page.extract_words():
            text = element['text']
            x0, y0, x1, y1 = element['x0'], element['top'], element['x1'], element['bottom']
            data.append([page_num + 1, text, x0, y0, x1, y1])

    # Convert the list to a DataFrame
    df = pd.DataFrame(data, columns=['Page', 'Text', 'X0', 'Y0', 'X1', 'Y1'])
    # Save the DataFrame to a CSV file
    df.to_csv('extracted_text.csv', index=False)


gridlines = {
    52.92000: "plate",
    121.39472: "vrm",
    220.56000: "make",
    312.33007: "model",
    414.67504: "license_start",
    478.32001: "license_end"
}


Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12


In [2]:

gridlines = {
    53: "plate",
    121: "vrm",
    221: "make",
    312: "model",
    419: "license_start",
    483: "license_end"
}

first_value = "PHV3429"

df["value"] = df['X0'].apply(lambda x: gridlines.get(round(x, 0), None))
df["value"] = df["value"].fillna(method='ffill')

/var/folders/8j/qdfkl38s5p11l966skbxw4300000gn/T/ipykernel_73866/3951864876.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["value"] = df["value"].fillna(method='ffill')


In [3]:
df[24:].to_csv("test.csv")

In [4]:
df_reduced = df[24:][["Text", "value"]].reset_index(drop=True)

In [5]:
df_reduced.to_csv("df_reduced.csv", index=False)

In [8]:
df_reduced

,Text,value
0,PHV3429,plate
1,SF64,vrm
2,FVE,vrm
3,Peugeot,make
4,Independence,model
...,...,...
4410,ATV,vrm
4411,Peugeot,make
4412,Horizon,model
4413,10/02/2024,license_start


In [9]:
new_value_column = []
new_text_column = []

# Iterate over the dataframe to concatenate values
current_value = None
current_text = ""

for value, text in zip(df_reduced['value'], df_reduced['Text']):
    if value == current_value:
        current_text += text + " "
    else:
        if current_value is not None:
            new_value_column.append(current_value)
            new_text_column.append(current_text.strip())
        current_value = value
        current_text = text

# Append the last accumulated values
if current_value is not None:
    new_value_column.append(current_value)
    new_text_column.append(current_text.strip())

# Create a new DataFrame with the concatenated values
new_df = pd.DataFrame({
    'value': new_value_column,
    'text': new_text_column
})

In [7]:
new_df.to_csv("inspection.csv")